## 2d. Evidence - Interpretability QAS Measurements.

Now we proceed to gather data about the Interpretability of the model, for the corresponding scenario.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.scenarios.session import *

### Helper Functions

Functions to help, and data setup.

In [ ]:
from demo.scenarios.model_analysis import *

# Load the model/
loaded_model = load_model(MODEL_FILE_PATH)

In [ ]:
# Load and show the image.

flower_img = "flower3.jpg"  # Filename of flower image to use, public domain image adapted from: https://commons.wikimedia.org/wiki/File:Beautiful_white_flower_in_garden.jpg
flower_idx = (
    42  # Classifier index of associated flower (see OxfordFlower102Labels.csv)
)

im = read_image(os.path.join(SAMPLE_DATASET_DIR, flower_img))

plt.imshow(im)
plt.axis("off")
plt.show()

In [ ]:
predictions = run_model(im, loaded_model)

baseline, alphas = generate_baseline_and_alphas()

In [ ]:
interpolated_images = interpolate_images(
    baseline=baseline, image=im, alphas=alphas
)

In [ ]:
fig = plt.figure(figsize=(20, 20))

i = 0
for alpha, image in zip(alphas[0::10], interpolated_images[0::10]):
    i += 1
    plt.subplot(1, len(alphas[0::10]), i)
    plt.title(f"alpha: {alpha:.1f}")
    plt.imshow(image)
    plt.axis("off")

plt.tight_layout()

In [ ]:
path_gradients = compute_gradients(
    loaded_model=loaded_model,
    images=interpolated_images,
    target_class_idx=flower_idx,
)
print(path_gradients.shape)

ig = integral_approximation(gradients=path_gradients)
print(ig.shape)

In [ ]:
ig_attributions = integrated_gradients(
    baseline=baseline,
    image=im,
    target_class_idx=flower_idx,
    loaded_model=loaded_model,
    m_steps=240,
)
print(ig_attributions.shape)

### Measurements

Execute and store measurements.

In [ ]:
fig = plot_img_attributions(
    image=im,
    baseline=baseline,
    target_class_idx=flower_idx,
    loaded_model=loaded_model,
    m_steps=240,
    cmap=plt.cm.inferno,
    overlay_alpha=0.4,
)

plt.savefig(MEDIA_DIR / "attributions.png")

In [ ]:
from mlte.measurement.external_measurement import ExternalMeasurement
from mlte.evidence.types.image import Image

# Save to MLTE store.
img_collector = ExternalMeasurement("image attributions", Image)
img = img_collector.ingest(MEDIA_DIR / "attributions.png")
img.save(force=True)